In [1]:
from __future__ import print_function

import allfunctions as func
import tensorflow as tf
import numpy as np
import keras
import nltk
import string


from nltk.tokenize import word_tokenize, RegexpTokenizer
from scipy.spatial.distance import cdist
from keras.models import Sequential, load_model
from keras.layers import Dense, GRU, Embedding, Dropout, LSTM, Bidirectional
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras import regularizers
from keras.regularizers import l2
from matplotlib import pyplot

#required download for pos_tagger to work
#nltk.download('averaged_perceptron_tagger') 

max_features = 10000 # (Use the "max_features" most common words as features)
embed_len = 16 # How long is each embedding vector
INDEX_FROM= 3 #Where does word indexing start
maxlen = 234 #234 average lenght of review in set
batch_size = 1024 ## 4096 causes accuracy problems due to gradiant problems
                  ## 512 or 1024 has best results with time used and accuracy
(x_traintext, y_trainvalue), (x_testtext, y_testvalue) = imdb.load_data(num_words=max_features, index_from=INDEX_FROM,skip_top=0)


Using TensorFlow backend.


In [2]:
number_of_tokens = [len(i) for i in x_traintext + x_testtext]
print("Average length:" ,np.mean(number_of_tokens) / 2) 
#dividing by two because of way I'm adding x_traintext and x_testtext together here


Average length: 234.75892


In [3]:

##https://stackoverflow.com/questions/42821330/restore-original-text-from-keras-s-imdb-dataset
word_to_id = keras.datasets.imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
word_to_id["<DEL>"] = 3
## 0-3 is for custom use index 4 is the first word
id_to_word = {value:key for key,value in word_to_id.items()} ##tokenizer.word_index()
inverse_map = dict(zip(word_to_id.values(), word_to_id.keys())) ##used in assessing vector distance in the embedding layer

#Cut out 5000 reviews and their sentiment value
x_remove_text = x_testtext[20000:]
y_remove_sentiment = y_testvalue[20000:]
x_testtext = np.delete(x_testtext, np.s_[20000:], axis = 0)
y_testvalue = np.delete(y_testvalue, np.s_[20000:], axis = 0)

# turn all sets into actual words
x_trainfulltext = func.set_to_text(x_traintext,id_to_word)
x_testfulltext = func.set_to_text(x_testtext,id_to_word)

## x_all_fullremove_text has the reviews as is, if needed later for comparison
x_all_fullremove_text = func.set_to_text(x_remove_text,id_to_word) 

x_fullremove_text = func.pos_tagger(x_all_fullremove_text)
x_fullremove_text = func.text_to_id(x_fullremove_text,word_to_id)

#All the pos_tagging is done before padding and cutting so the pos_tagger has full context
x_traintext = sequence.pad_sequences(x_traintext, maxlen=maxlen)
x_testtext = sequence.pad_sequences(x_testtext, maxlen=maxlen)
x_remove_text = sequence.pad_sequences(x_fullremove_text, maxlen=maxlen)





In [4]:
#model = load_model('model_1.h5')
#Uncomment if you are training your own model

model = Sequential()
model.add(Embedding(max_features, embed_len, input_length=maxlen,name="layer_embedding"))
model.add(LSTM(32, return_sequences=True, recurrent_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(LSTM(16, recurrent_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


'\nmodel = Sequential()\nmodel.add(Embedding(max_features, embed_len, input_length=maxlen,name="layer_embedding"))\nmodel.add(LSTM(32, return_sequences=True, recurrent_regularizer=l2(0.01)))\nmodel.add(Dropout(0.3))\nmodel.add(LSTM(16, recurrent_regularizer=l2(0.01)))\nmodel.add(Dropout(0.3))\nmodel.add(Dense(1, activation=\'sigmoid\'\nmodel.compile(optimizer=\'adam\', loss=\'binary_crossentropy\', metrics=[\'accuracy\'])\n'

In [5]:
#uncomment if you are training you own model

mo = model.fit(x_traintext, y_trainvalue,
          batch_size=batch_size,
          epochs=6,
          validation_data=[x_testtext, y_testvalue])

pyplot.plot(mo.history['loss'], label='train')
pyplot.plot(mo.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()
#Save your own model
#model.save("model_2.h5")


'\nmo = model.fit(x_traintext, y_trainvalue,\n          batch_size=batch_size,\n          epochs=6,\n          validation_data=[x_testtext, y_testvalue])\n\npyplot.plot(mo.history[\'loss\'], label=\'train\')\npyplot.plot(mo.history[\'val_loss\'], label=\'test\')\npyplot.legend()\npyplot.show()\n#Save your own model\n#model.save("model_2.h5")\n'

In [6]:
#Evaluate model with full context
testresults = model.evaluate(x_testtext, y_testvalue,verbose=0)
print("Base accuracy",testresults[1])
#Evaluate model with word classes removed
evaluate = model.evaluate(x_remove_text, y_remove_sentiment,verbose=0)
print("Evaluation accuracy",evaluate[1])


Base accuracy 0.8689
Evaluation accuracy 0.65


# #EVERYTHING UNDER HERE IS NOT REALLY RELEVANT CURRENTLY

In [8]:
#https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/20_Natural_Language_Processing.ipynb
y_value = model.predict(x_remove_text[0:1000])
y_value = y_value.T[0]
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_value])
cls_true = np.array(y_remove_sentiment[0:1000])
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]


index = incorrect[2]
print(len(incorrect))
func.texter(x_remove_text,index,id_to_word)
print("Model Prediction:",y_value[index])
print("Actual Sentiment:",y_remove_sentiment[index])

344
<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <START> <DEL> <DEL> <DEL> <UNK> <DEL> <DEL> us a <DEL> film a simple but <DEL> story <DEL> in a <DEL> paradise and the music can't <DEL> <DEL> stephen <UNK> <UNK> us <DEL> <DEL> and <DEL> <UNK> but why on earth <DEL> <DEL> <UNK> in this film i <DEL> myself that <DEL> question while <DEL> this movie of course her <DEL> accent <DEL> <DEL> <DEL> she's of <DEL> <UNK> <DEL> <DEL> in one scene <DEL> <DEL> at the end she <DEL> a <DEL> job after <DEL> <DEL> events after years she <DEL> <DEL> to <DEL> <DEL> and she <DEL> <DEL> the <DEL> face of we <DEL> to <DEL> during the <DEL> film <DEL> cage <DEL> <DEL> <DEL> in one of his <DEL> characters hurt <DEL> <DEL> <DEL> in his 

In [ ]:
def print_sorted_words(word, metric='cosine'):
    #https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/20_Natural_Language_Processing.ipynb
    layer_embedding = model.get_layer('layer_embedding')
    weights_embedding = layer_embedding.get_weights()[0]
    """
    Print the words in the vocabulary sorted according to their
    embedding-distance to the given word.
    Different metrics can be used, e.g. 'cosine' or 'euclidean'.
    """
   
    # Get the token (i.e. integer ID) for the given word.
    token = word_to_id[word]
    
    # Get the embedding for the given word. Note that the
    # embedding-weight-matrix is indexed by the word-tokens
    # which are integer IDs.
    embedding = weights_embedding[token]
    
    # Calculate the distance between the embeddings for
    # this word and all other words in the vocabulary.
    distances = cdist(weights_embedding, [embedding],
                      metric=metric).T[0]
    
    # Get an index sorted according to the embedding-distances.
    # These are the tokens (integer IDs) for words in the vocabulary.
    sorted_index = np.argsort(distances)
   
    # Sort the embedding-distances.
    sorted_distances = distances[sorted_index]
    
    # Sort all the words in the vocabulary according to their
    # embedding-distance. This is a bit excessive because we
    # will only print the top and bottom words.
    sorted_words = [inverse_map[token] for token in sorted_index if token != 0]

    # Helper-function for printing words and embedding-distances.
    def _print_words(words, distances):
        for word, distance in zip(words, distances):
            print("{0:.3f} - {1}".format(distance, word))

    # Number of words to print from the top and bottom of the list.
    k = 10

    print("Distance from '{0}':".format(word))

    # Print the words with smallest embedding-distance.
    _print_words(sorted_words[0:k], sorted_distances[0:k])

    print("...")

    # Print the words with highest embedding-distance.
    _print_words(sorted_words[-k:], sorted_distances[-k:])

print_sorted_words('great', metric='cosine')